In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import requests 
from bs4 import BeautifulSoup 


In [ ]:
# pull data from google sheets
def sheetsync(datasheet,tab):
    
    scope = ['https://spreadsheets.google.com/feeds',
             'https://www.googleapis.com/auth/drive']
    credentials = ServiceAccountCredentials.from_json_keyfile_name(
             'qd-cnov-database-2027ad01e8de.json', scope) 
    gc = gspread.authorize(credentials)
    
    spreadsheetName = datasheet
    sheetName = tab  # <--- please set the sheet name here.
    spreadsheet = gc.open(spreadsheetName)
    sheet = spreadsheet.worksheet(sheetName)
    
    data = sheet.get_all_values()
    headers = data.pop(0)

    df = pd.DataFrame(data, columns=headers)
    return df
sheetsync('dataset','qd')

### crawler

In [144]:

def start_requests(url):
    print(url) 
    r = requests.get(url)
    return r.content

# 第一级 links 
def get_page(text):
    soup = BeautifulSoup(text, 'html.parser')
    links = soup.find_all('a', class_ = 'nrf')
    pages = []
    for link in links:
        url = 'http://wsjsw.qingdao.gov.cn/'+link.get('href')
        pages.append(url)
    return pages

# 第二级 links and get info
def parse_page(text):
    soup = BeautifulSoup(text, 'html.parser')
    mydict = {}
    mydict['time'] = soup.find('div', class_ = 'fbbt-wenz').text
    mydict['parag'] = soup.find('div', class_ = 'neirong-wz').text
    return mydict


def write_json(result):
    s = json.dumps(result, indent = 4, ensure_ascii=False)
    with open('data/bulletin.json', 'w', encoding = 'utf-8') as f:
        f.write(s)
def crawl(url):
    text = start_requests(url)
    pageurls = get_page(text) 
    for pageurl in pageurls: 
        page = start_requests(pageurl)
        mydict = parse_page(page)
        result_list.append(mydict)
def main():
    url = 'http://wsjsw.qingdao.gov.cn/n28356065/n32563060/n32563061/index.html'
    crawl(url)
    for i in range(2, 4):
        url = 'http://wsjsw.qingdao.gov.cn/n28356065/n32563060/n32563061/index_{}.html'.format(i)
        crawl(url)
#         text = start_requests(url)
#         pageurls = get_page(text) 
#         for pageurl in pageurls: # 
#             page = start_requests(pageurl)
#             mydict = parse_page(page)
#             result_list.append(mydict)
    write_json(result_list) 

if __name__ == '__main__':
    result_list = []
    main()

http://wsjsw.qingdao.gov.cn/n28356065/n32563060/n32563061/index.html
http://wsjsw.qingdao.gov.cn//n28356065/n32563060/n32563061/200204155030655835.html
http://wsjsw.qingdao.gov.cn//n28356065/n32563060/n32563061/200204092147343110.html
http://wsjsw.qingdao.gov.cn//n28356065/n32563060/n32563061/200203144006609133.html
http://wsjsw.qingdao.gov.cn//n28356065/n32563060/n32563061/200203092831140802.html
http://wsjsw.qingdao.gov.cn//n28356065/n32563060/n32563061/200202152750421844.html
http://wsjsw.qingdao.gov.cn//n28356065/n32563060/n32563061/200202102259593635.html
http://wsjsw.qingdao.gov.cn//n28356065/n32563060/n32563061/200202101056984886.html
http://wsjsw.qingdao.gov.cn//n28356065/n32563060/n32563061/200201155219671331.html
http://wsjsw.qingdao.gov.cn//n28356065/n32563060/n32563061/200201110700218168.html
http://wsjsw.qingdao.gov.cn//n28356065/n32563060/n32563061/200131154113984447.html
http://wsjsw.qingdao.gov.cn//n28356065/n32563060/n32563061/200131103115187746.html
http://wsjsw.qingd

In [114]:
with open('data/bulletin.json',encoding="utf8") as f:
    data = json.load(f)
data

[{'time': ' \xa0 \xa0\xa0\xa0  2020-02-04',
  'parag': '\n2020年2月4日0—12时，我市新增3例新型冠状病毒感染的肺炎确诊病例。新增3例确诊患者，一为20岁男性，在武汉上大学，1月21日从武汉返回青岛，现居城阳区，2月2日到医院发热门诊就诊；一为63岁女性，为2月3日确诊病例37岁男性患者的直系亲属，1月24日由湖北荆州自驾抵青探亲，现居即墨区，2月3日到医院发热门诊就诊；一为85岁女性，为2月3日确诊病例28岁男性患者的直系亲属，自1月23日起在市南区某医院住院治疗。以上3名患者经市疾控中心核酸检测和专家组评估，2月4日上午确诊。现已对15名密切接触者开展医学观察，目前无发热等异常情况。其他密切接触者正在进一步追踪和调查中。截至2月4日12时，我市累计确诊32例新型冠状病毒感染的肺炎病例，其中重症1例，危重症2例，治愈出院1例，患者均在定点医院接受隔离治疗。累计已排除疑似病例79例，现有疑似病例20例，正在隔离检测治疗。累计已解除医学观察123人，仍纳入医学观察的密切接触者299人，未发现异常。目前按病例确诊医院所在区市分布：市南区3例（已治愈出院1例）、市北区4例、李沧区2例、崂山区2例、城阳区2例、黄岛区9例、即墨区5例、胶州市1例、平度市3例、莱西市1例。\xa0\xa0\xa0\xa0\xa0\xa0 \xa0青岛市卫生健康委员会2020年2月4日 \n'},
 {'time': ' \xa0 \xa0\xa0\xa0  2020-02-04',
  'parag': '\n2020年2月3日12—24时，我市新增3例新型冠状病毒感染的肺炎确诊病例。新增3例确诊患者,一为市北区28岁男性，1月24日参加过亲友聚餐，1月25日出现发热等症状，2月2日到市南区某医院就诊，经预检分诊在发热门诊隔离治疗；一为37岁男性，1月24日由湖北荆州自驾抵青探亲，现居即墨区，2月3日到医院发热门诊就诊；一为平度市70岁女性，为1月29日确诊病例的密切接触者，2月3日到医院发热门诊就诊。以上3名患者经市疾控中心核酸检测和专家组评估，2月3日下午确诊。现已对14名密切接触者开展医学观察，目前无发热等异常情况。其他密切接触者正在进一步追踪和调查中。截至2月3日24时，我市累计确诊29例新型冠状病毒感染的肺炎病例，其中重

In [147]:
df = pd.read_json('data/bulletin.json',encoding="utf8")
df.head()

,time,parag
0,2020-02-04,\n2020年2月4日0—12时，我市新增3例新型冠状病毒感染的肺炎确诊病例。新增3例确诊患...
1,2020-02-04,\n2020年2月3日12—24时，我市新增3例新型冠状病毒感染的肺炎确诊病例。新增3例确诊...
2,2020-02-03,\n2020年2月3日0—12时，我市新增1例新型冠状病毒感染的肺炎确诊病例。新增确诊患者，...
3,2020-02-03,\n2020年2月2日12—24时，我市新增1例新型冠状病毒感染的肺炎确诊病例。新增确诊患者...
4,2020-02-02,\n2020年2月2日0—12时，我市新增2例新型冠状病毒感染的肺炎确诊病例。2例新增确诊患...


In [141]:
######### TODO check if valid 
test = df.parag[0].split('分布：')[1].split('。')[0]
test

'市南区3例（已治愈出院1例）、市北区4例、李沧区2例、崂山区2例、城阳区2例、黄岛区9例、即墨区5例、胶州市1例、平度市3例、莱西市1例'

In [134]:
# list of diagnosed data 
import re
temp = re.findall(r'\d+', test) 
res = list(map(int, temp))
res

[3, 1, 4, 2, 2, 2, 9, 5, 1, 3, 1]